# * VINSIGHT : Inflow

## Parameter

In [2]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Import Transaction

In [3]:
''' Execute transaction '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B0R00010001CS' --Total Inflow M1
        , 'TB0R00010001CS' --Total Inflow M1 : TRUE
        , 'DB0R00010001CS' --Total Inflow M1 : DTAC
        , 'B0R00010001CG' --Total Inflow M1 - GEO Channel
        , 'TB0R00010001CG' --Total Inflow M1 : TRUE - GEO Channel
        , 'DB0R00010001CG' --Total Inflow M1 : DTAC - GEO Channel
        
        , 'B1R000900CS' --Prepaid Inflow M1
        , 'TB1R000900CS' --Prepaid Inflow M1 : TMH
        , 'DB1R000900CS' --Prepaid Inflow M1 : DTAC
        , 'B1R000900CG' --Prepaid Inflow M1 - GEO Channel
        , 'TB1R000900CG' --Prepaid Inflow M1 : TMH - GEO Channel
        , 'DB1R000900CG' --Prepaid Inflow M1 : DTAC - GEO Channel
        
        , 'B2R010500CS' --Postpaid Inflow M1 B2C
        , 'TB2R010500CS' --Postpaid Inflow M1 B2C : TMH
        , 'DB2R010500CS' --Postpaid Inflow M1 B2C : DTAC
        , 'B2R010500CG' --Postpaid Inflow M1 B2C - GEO Channel
        , 'TB2R010500CG' --Postpaid Inflow M1 B2C : TMH - GEO Channel
        , 'DB2R010500CG' --Postpaid Inflow M1 B2C : DTAC - GEO Channel
        , 'B2R020500CS' --Postpaid Inflow M1 B2B
        , 'TB2R020500CS' --Postpaid Inflow M1 B2B : TMH
        , 'DB2R020500CS' --Postpaid Inflow M1 B2B : DTAC
        , 'B2R020500CG' --Postpaid Inflow M1 B2B - GEO Channel
        , 'DB2R020500CG' --Postpaid Inflow M1 B2B : DTAC - GEO Channel
        , 'TB2R020500CG' --Postpaid Inflow M1 B2B : TMH - GEO Channel
        
        , 'TB3R000601CS' --TOL Inflow M1 Connected : Consumer
        , 'TB3R000601CG' --TOL Inflow M1 Connected : Consumer - GEO Channel
        , 'TB3R000601D1CS' --TOL Inflow M1 Connected : Consumer (Install Location)
        , 'TB3R000601D1CG' --TOL Inflow M1 Connected : Consumer - GEO Channel (Install Location)
        , 'TB3R000602CS' --TOL Inflow M1 Connected : DataService
        , 'TB3R000602CG' --TOL Inflow M1 Connected : DataService - GEO Channel
        , 'TB3R000602D1CS' --TOL Inflow M1 Connected : DataService (Install Location)
        , 'TB3R000602D1CG' --TOL Inflow M1 Connected : DataService - GEO Channel (Install Location)

        , 'TB4R001700CS' --TVS Now Inflow M1
        , 'TB4R001700CG' --TVS Now Inflow M1 - GEO Channel
        , 'TB4R001004CS' --TVS CMDU Inflow M1
        , 'TB4R001004CG' --TVS CMDU Inflow M1 - GEO Channel
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    --AND TM_KEY_DAY IN (20240131, 20240229, 20240331, 20240430, 20240531, 20240630, 20240731, 20240831, 20240930, 20241031, 20241130, 20241231, 20250131, 20250228, 20250331) -->> Year 2024
    --AND TM_KEY_DAY IN (20250131, 20250228, 20250331, 20250430, 20250531) -->> Year 2025
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
    --ORDER BY TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2025-06-02, 17:22:00
   -> v_start_date: 20250101


In [4]:
''' Automate Currently Period '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = str(chk_src_df['TM_KEY_MTH'].drop_duplicates().astype(int).shift().max())

## Daily (All & Geo)

In [12]:
''' Inflow M1(All Channel) Daily '''

v_metric_list = [
    'B0R00010001CS' #Total Inflow M1
    , 'TB0R00010001CS' #Total Inflow M1 : TRUE
    , 'DB0R00010001CS' #Total Inflow M1 : DTAC

    , 'B1R000900CS' #Prepaid Inflow M1
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC

    , 'B2R010500CS' #Postpaid Inflow M1 B2C
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    # , 'B2R020500CS' #Postpaid Inflow M1 B2B
    # , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    # , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC

    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    , 'TB3R000601D1CS' #TOL Inflow M1 Connected : Consumer (Install Location)
    # , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    # , 'TB3R000602D1CS' #TOL Inflow M1 Connected : DataService (Install Location)

    , 'TB4R001700CS' #TVS Now Inflow M1
    # , 'TB4R001004CS' #TVS CMDU Inflow M1
    ]

# v_tm_key_mth = curr_mth
v_tm_key_mth = 202505

m1_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
m1_daily_df = m1_daily_df.loc[m1_daily_df['METRIC_CD'].isin(v_metric_list)]
m1_daily_df = m1_daily_df.loc[m1_daily_df['TM_KEY_MTH']==v_tm_key_mth]

m1_daily_df['TOTAL'] = np.where(m1_daily_df['METRIC_CD']=='B0R00010001CS', m1_daily_df['P'], 0)
m1_daily_df['TOTAL_T'] = np.where(m1_daily_df['METRIC_CD']=='TB0R00010001CS', m1_daily_df['P'], 0)
m1_daily_df['TOTAL_D'] = np.where(m1_daily_df['METRIC_CD']=='DB0R00010001CS', m1_daily_df['P'], 0)
m1_daily_df['PRE'] = np.where(m1_daily_df['METRIC_CD']=='B1R000900CS', m1_daily_df['P'], 0)
m1_daily_df['PRE_T'] = np.where(m1_daily_df['METRIC_CD']=='TB1R000900CS', m1_daily_df['P'], 0)
m1_daily_df['PRE_D'] = np.where(m1_daily_df['METRIC_CD']=='DB1R000900CS', m1_daily_df['P'], 0)
m1_daily_df['POST'] = np.where(m1_daily_df['METRIC_CD']=='B2R010500CS', m1_daily_df['P'], 0)
m1_daily_df['POST_T'] = np.where(m1_daily_df['METRIC_CD']=='TB2R010500CS', m1_daily_df['P'], 0)
m1_daily_df['POST_D'] = np.where(m1_daily_df['METRIC_CD']=='DB2R010500CS', m1_daily_df['P'], 0)
m1_daily_df['TOL'] = np.where(m1_daily_df['METRIC_CD']=='TB3R000601CS', m1_daily_df['P'], 0)
m1_daily_df['TOL(Install)'] = np.where(m1_daily_df['METRIC_CD']=='TB3R000601D1CS', m1_daily_df['P'], 0)
m1_daily_df['TVS_NOW'] = np.where(m1_daily_df['METRIC_CD']=='TB4R001700CS', m1_daily_df['P'], 0)

m1_daily_df = m1_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST':'sum', 'POST_T':'sum', 'POST_D':'sum', 'TOL':'sum', 'TOL(Install)':'sum', 'TVS_NOW':'sum'})
m1_daily_df = m1_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY']).reset_index()
m1_daily_df = m1_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST', 'POST_T', 'POST_D', 'TOL', 'TOL(Install)', 'TVS_NOW']]

mod_col_list = m1_daily_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_daily_df[col] = m1_daily_df[col].apply(lambda x: format(x, ',.0f'))
m1_daily_df

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST,POST_T,POST_D,TOL,TOL(Install),TVS_NOW
0,202505,20250501,2025-06-02 04:05:56,"19,313,632","9,545,658","9,767,974","15,545,611","6,691,969","8,853,642","3,462,358","2,568,324","894,034","39,405","39,405","45,415"
1,202505,20250502,2025-06-02 04:05:56,"20,482,623","10,040,308","10,442,315","13,455,125","6,455,483","6,999,641","2,783,368","2,202,384","580,984","1,025,713","1,025,713","40,262"
2,202505,20250503,2025-06-02 04:05:56,"16,129,269","9,616,036","6,513,233","11,776,980","5,861,354","5,915,627","3,079,848","2,491,302","588,546","1,084,907","1,084,907","22,483"
3,202505,20250504,2025-06-02 04:05:56,"15,427,128","9,160,486","6,266,642","11,159,782","5,414,987","5,744,795","3,256,681","2,742,718","513,963","962,804","962,804","29,449"
4,202505,20250505,2025-06-02 04:05:56,"15,575,094","9,133,261","6,441,833","11,483,685","5,722,895","5,760,790","2,946,041","2,272,444","673,597","1,050,423","1,050,423","35,699"
5,202505,20250506,2025-06-02 04:05:56,"17,628,809","9,290,442","8,338,367","12,003,919","5,760,340","6,243,578","2,959,418","2,209,869","749,549","1,073,260","1,073,260","25,776"
6,202505,20250507,2025-06-02 04:05:56,"18,000,824","10,017,895","7,982,929","12,424,043","6,336,293","6,087,750","3,078,304","2,353,020","725,284","1,039,126","1,039,126","9,303"
7,202505,20250508,2025-06-02 04:05:56,"16,405,736","9,735,255","6,670,481","12,474,491","6,518,341","5,956,150","2,563,864","1,921,726","642,138","1,010,280","1,010,280","13,494"
8,202505,20250509,2025-06-02 04:05:56,"17,408,585","10,331,790","7,076,795","12,806,570","6,574,438","6,232,133","2,716,967","1,957,048","759,919","1,028,909","1,028,909","26,830"
9,202505,20250510,2025-06-02 04:05:56,"16,544,600","9,301,498","7,243,103","12,933,745","6,415,872","6,517,874","2,605,080","1,904,727","700,353","906,106","906,106","20,424"


In [13]:
''' Inflow M1(Geo Channel) Daily '''

v_metric_list = [
    'B0R00010001CG' #Total Inflow M1 - GEO Channel
    , 'TB0R00010001CG' #Total Inflow M1 : TRUE - GEO Channel
    , 'DB0R00010001CG' #Total Inflow M1 : DTAC - GEO Channel

    , 'B1R000900CG' #Prepaid Inflow M1 - GEO Channel
    , 'TB1R000900CG' #Prepaid Inflow M1 : TMH - GEO Channel
    , 'DB1R000900CG' #Prepaid Inflow M1 : DTAC - GEO Channel
    
    , 'B2R010500CG' #Postpaid Inflow M1 B2C - GEO Channel
    , 'TB2R010500CG' #Postpaid Inflow M1 B2C : TMH - GEO Channel
    , 'DB2R010500CG' #Postpaid Inflow M1 B2C : DTAC - GEO Channel
    # , 'B2R020500CG' #Postpaid Inflow M1 B2B - GEO Channel
    # , 'DB2R020500CG' #Postpaid Inflow M1 B2B : DTAC - GEO Channel
    # , 'TB2R020500CG' #Postpaid Inflow M1 B2B : TMH - GEO Channel
    
    , 'TB3R000601CG' #TOL Inflow M1 Connected : Consumer - GEO Channel
    , 'TB3R000601D1CG' #TOL Inflow M1 Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3R000602CG' #TOL Inflow M1 Connected : DataService - GEO Channel
    # , 'TB3R000602D1CG' #TOL Inflow M1 Connected : DataService - GEO Channel (Install Location)

    , 'TB4R001700CG' #TVS Now Inflow M1 - GEO Channel
    # , 'TB4R001004CG' #TVS CMDU Inflow M1 - GEO Channel
    ]

# v_tm_key_mth = curr_mth
v_tm_key_mth = 202505

m1_geo_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
m1_geo_daily_df = m1_geo_daily_df.loc[m1_geo_daily_df['METRIC_CD'].isin(v_metric_list)]
m1_geo_daily_df = m1_geo_daily_df.loc[m1_geo_daily_df['TM_KEY_MTH']==v_tm_key_mth]

m1_geo_daily_df['TOTAL'] = np.where(m1_geo_daily_df['METRIC_CD']=='B0R00010001CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['TOTAL_T'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB0R00010001CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['TOTAL_D'] = np.where(m1_geo_daily_df['METRIC_CD']=='DB0R00010001CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['PRE'] = np.where(m1_geo_daily_df['METRIC_CD']=='B1R000900CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['PRE_T'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB1R000900CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['PRE_D'] = np.where(m1_geo_daily_df['METRIC_CD']=='DB1R000900CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['POST'] = np.where(m1_geo_daily_df['METRIC_CD']=='B2R010500CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['POST_T'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB2R010500CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['POST_D'] = np.where(m1_geo_daily_df['METRIC_CD']=='DB2R010500CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['TOL'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB3R000601CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['TOL(Install)'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB3R000601D1CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['TVS_NOW'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB4R001700CG', m1_geo_daily_df['P'], 0)

m1_geo_daily_df = m1_geo_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST':'sum', 'POST_T':'sum', 'POST_D':'sum', 'TOL':'sum', 'TOL(Install)':'sum', 'TVS_NOW':'sum'})
m1_geo_daily_df = m1_geo_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY']).reset_index()
m1_geo_daily_df = m1_geo_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST', 'POST_T', 'POST_D', 'TOL', 'TOL(Install)', 'TVS_NOW']]

mod_col_list = m1_geo_daily_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_geo_daily_df[col] = m1_geo_daily_df[col].apply(lambda x: format(x, ',.0f'))
m1_geo_daily_df

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST,POST_T,POST_D,TOL,TOL(Install),TVS_NOW
0,202505,20250501,2025-06-02 04:05:56,"17,265,066","8,133,806","9,131,261","15,321,796","6,618,108","8,703,688","1,774,294","1,367,019","407,275","34,465","34,465","45,415"
1,202505,20250502,2025-06-02 04:05:56,"15,718,484","8,550,130","7,168,354","13,280,825","6,385,230","6,895,595","1,448,565","1,181,964","266,601","844,641","844,641","40,262"
2,202505,20250503,2025-06-02 04:05:56,"13,969,721","7,904,556","6,065,165","11,595,731","5,781,359","5,814,372","1,459,429","1,217,247","242,182","868,607","868,607","22,483"
3,202505,20250504,2025-06-02 04:05:56,"13,471,881","7,469,807","6,002,074","11,022,487","5,367,988","5,654,498","1,642,591","1,302,899","339,692","759,044","759,044","29,449"
4,202505,20250505,2025-06-02 04:05:56,"13,631,335","7,705,630","5,925,705","11,357,524","5,681,066","5,676,458","1,343,882","1,102,081","241,801","855,506","855,506","35,476"
5,202505,20250506,2025-06-02 04:05:56,"14,460,844","8,013,356","6,447,488","11,890,097","5,704,545","6,185,553","1,508,030","1,257,043","250,987","903,968","903,968","25,776"
6,202505,20250507,2025-06-02 04:05:56,"14,860,187","8,564,030","6,296,157","12,275,011","6,260,553","6,014,457","1,631,298","1,359,178","272,119","858,771","858,771","9,303"
7,202505,20250508,2025-06-02 04:05:56,"14,432,928","8,317,316","6,115,612","12,282,876","6,398,658","5,884,218","1,225,513","1,010,828","214,685","822,598","822,598","13,494"
8,202505,20250509,2025-06-02 04:05:56,"14,895,024","8,485,110","6,409,914","12,649,947","6,488,425","6,161,522","1,168,090","929,677","238,413","835,790","835,790","26,830"
9,202505,20250510,2025-06-02 04:05:56,"14,687,343","7,995,456","6,691,888","12,783,524","6,342,218","6,441,307","1,149,514","919,912","229,602","696,617","696,617","20,424"


In [ ]:
# WHERE METRIC_CD IN (
#     'B0R00010001CS' --Total Inflow M1
#     , 'TB0R00010001CS' --Total Inflow M1 : TRUE
#     , 'DB0R00010001CS' --Total Inflow M1 : DTAC
#     , 'B0R00010001CG' --Total Inflow M1 - GEO Channel
#     , 'TB0R00010001CG' --Total Inflow M1 : TRUE - GEO Channel
#     , 'DB0R00010001CG' --Total Inflow M1 : DTAC - GEO Channel
    
#     , 'B1R000900CS' --Prepaid Inflow M1
#     , 'TB1R000900CS' --Prepaid Inflow M1 : TMH
#     , 'DB1R000900CS' --Prepaid Inflow M1 : DTAC
#     , 'B1R000900CG' --Prepaid Inflow M1 - GEO Channel
#     , 'TB1R000900CG' --Prepaid Inflow M1 : TMH - GEO Channel
#     , 'DB1R000900CG' --Prepaid Inflow M1 : DTAC - GEO Channel
    
#     , 'B2R010500CS' --Postpaid Inflow M1 B2C
#     , 'TB2R010500CS' --Postpaid Inflow M1 B2C : TMH
#     , 'DB2R010500CS' --Postpaid Inflow M1 B2C : DTAC
#     , 'B2R010500CG' --Postpaid Inflow M1 B2C - GEO Channel
#     , 'TB2R010500CG' --Postpaid Inflow M1 B2C : TMH - GEO Channel
#     , 'DB2R010500CG' --Postpaid Inflow M1 B2C : DTAC - GEO Channel
#     , 'B2R020500CS' --Postpaid Inflow M1 B2B
#     , 'TB2R020500CS' --Postpaid Inflow M1 B2B : TMH
#     , 'DB2R020500CS' --Postpaid Inflow M1 B2B : DTAC
#     , 'B2R020500CG' --Postpaid Inflow M1 B2B - GEO Channel
#     , 'DB2R020500CG' --Postpaid Inflow M1 B2B : DTAC - GEO Channel
#     , 'TB2R020500CG' --Postpaid Inflow M1 B2B : TMH - GEO Channel
    
#     , 'TB3R000601CS' --TOL Inflow M1 Connected : Consumer
#     , 'TB3R000601CG' --TOL Inflow M1 Connected : Consumer - GEO Channel
#     , 'TB3R000601D1CS' --TOL Inflow M1 Connected : Consumer (Install Location)
#     , 'TB3R000601D1CG' --TOL Inflow M1 Connected : Consumer - GEO Channel (Install Location)
#     , 'TB3R000602CS' --TOL Inflow M1 Connected : DataService
#     , 'TB3R000602CG' --TOL Inflow M1 Connected : DataService - GEO Channel
#     , 'TB3R000602D1CS' --TOL Inflow M1 Connected : DataService (Install Location)
#     , 'TB3R000602D1CG' --TOL Inflow M1 Connected : DataService - GEO Channel (Install Location)

#     , 'TB4R001700CS' --TVS Now Inflow M1
#     , 'TB4R001700CG' --TVS Now Inflow M1 - GEO Channel
#     , 'TB4R001004CS' --TVS CMDU Inflow M1
#     , 'TB4R001004CG' --TVS CMDU Inflow M1 - GEO Channel
#     )